In [1]:
%load_ext sql
%sql postgresql://appdev@data/appdev

'Connected: appdev@appdev'

In [4]:
%sql select * from pg_indexes where tablename = 'circuits';

3 rows affected.


schemaname,tablename,indexname,tablespace,indexdef
f1db,circuits,idx_17102_url,None,CREATE UNIQUE INDEX idx_17102_url ON circuits USING btree (url)
f1db,circuits,circuits_position_idx,None,"CREATE INDEX circuits_position_idx ON circuits USING gist (""position"")"
f1db,circuits,idx_17102_primary,None,CREATE UNIQUE INDEX idx_17102_primary ON circuits USING btree (circuitid)


Above we find all the indexnames

In [5]:
%sql select pg_size_pretty(pg_relation_size('idx_17102_url'));

1 rows affected.


pg_size_pretty
16 kB


In [6]:
%sql select pg_size_pretty(pg_relation_size('circuits_position_idx'));

1 rows affected.


pg_size_pretty
8192 bytes


In [8]:
%sql select pg_size_pretty(pg_relation_size('idx_17102_primary'));

1 rows affected.


pg_size_pretty
16 kB


Above we see the size of the 3 different indexes

In [36]:

%%sql
SELECT races.date, drivers.surname as "Driver's Surname", 
date_part('year', age('2018-03-19',dob)) as "Driver's Age", 
races.time as "Track Time(ms)", 
races.name as "Name", 
circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38
LIMIT 10;

10 rows affected.


date,Driver's Surname,Driver's Age,Track Time(ms),Name,Circuit Name
2008-03-16,Kovalainen,36.0,04:30:00,Australian Grand Prix,Albert Park Grand Prix Circuit
2008-04-06,Kovalainen,36.0,11:30:00,Bahrain Grand Prix,Bahrain International Circuit
2008-08-24,Massa,36.0,12:00:00,European Grand Prix,Valencia Street Circuit
2008-10-12,Massa,36.0,04:30:00,Japanese Grand Prix,Fuji Speedway
2008-10-19,Hamilton,33.0,07:00:00,Chinese Grand Prix,Shanghai International Circuit
2008-11-02,Massa,36.0,17:00:00,Brazilian Grand Prix,Autódromo José Carlos Pace
2007-04-08,Hamilton,33.0,07:00:00,Malaysian Grand Prix,Sepang International Circuit
2007-04-15,Massa,36.0,11:30:00,Bahrain Grand Prix,Bahrain International Circuit
2007-05-13,Massa,36.0,12:00:00,Spanish Grand Prix,Circuit de Barcelona-Catalunya
2007-05-27,Alonso,36.0,12:00:00,Monaco Grand Prix,Circuit de Monaco


In [31]:
%%sql
EXPLAIN ANALYZE
SELECT races.date, drivers.surname as "Driver's Surname", date_part('year', age('2018-03-19',dob)) as "Driver's Age", races.time as "Track Time(ms)", races.name as "Name", circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38;

22 rows affected.


QUERY PLAN
Hash Join (cost=77.92..792.94 rows=86 width=66) (actual time=1.438..7.960 rows=154 loops=1)
Hash Cond: (races.circuitid = circuits.circuitid)
-> Hash Join (cost=74.28..787.57 rows=86 width=50) (actual time=1.378..7.662 rows=154 loops=1)
Hash Cond: (results.raceid = races.raceid)
-> Hash Join (cost=34.32..746.53 rows=86 width=19) (actual time=0.768..6.966 rows=154 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=257 width=16) (actual time=0.010..6.104 rows=257 loops=1)
Filter: (rank = 1)
Rows Removed by Filter: 23420
-> Hash (cost=30.82..30.82 rows=280 width=19) (actual time=0.745..0.745 rows=60 loops=1)


Here we can see that there are 9 arrows, 
which means there are 10 calls made (the very first call does not have an arrow) 
from the analyse and the total
execution times varies a little bit, 
but seems to be around 8.000ms

In [34]:
%%sql CREATE MATERIALIZED VIEW race_winners_cache AS SELECT races.date, drivers.surname as "Driver's Surname", date_part('year', age('2018-03-19',dob)) as "Driver's Age", races.time as "Track Time(ms)", races.name as "Name", circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38;

154 rows affected.


[]

In [35]:
%sql 
EXPLAIN ANALYZE SELECT * FROM race_winners_cache;

3 rows affected.


QUERY PLAN
Seq Scan on race_winners_cache (cost=0.00..10.50 rows=50 width=1568) (actual time=0.012..0.094 rows=154 loops=1)
Planning time: 0.118 ms
Execution time: 0.127 ms


Above we can see how it works after it have been created in a materialized view and that it was much faster than the previous one.